# Feature Selection using the Adjusted Mutual Information (AMI) Score 

### (TCP analysis only)

This notebook uses the Adjusted Mutual Information Score to determine the most relevant features in the DDoS attacks. It uses the bigflows pcap from [http://tcpreplay.appneta.com/wiki/captures.html] containing normal network traffic. The DDoS attack pcaps from DDoSDB [https://ddosdb.org/] are used to compare it with the normal network traffic.

In [1]:
#all relevant libraries need to be imported
from sklearn.metrics.cluster import adjusted_mutual_info_score, mutual_info_score, normalized_mutual_info_score
from sklearn.feature_selection import mutual_info_classif
import numpy as np
import dpkt
import Utils
import ipaddress
import pandas as pd
import time
import os
import json
import operator

### Load bigflows containing only TCP traffic

In [2]:
start = time.time()

input = open('normal_pcaps/bigflows_tcp.pcap', 'rb')
normal = dpkt.pcap.Reader(input)
length = 633894 #this is the number of packets in bigflows_tcp.pcap   #791615 #791179

ips_n = np.empty(length)
protocol_n = np.empty(length)
total_length_n = np.empty(length)
src_port_n = np.empty(length)
dst_port_n = np.empty(length)
udp_length_n = np.empty(length)
header_length_n = np.empty(length)
urgent_pointer_n = np.empty(length)
window_size_n = np.empty(length)
flags_n = np.empty(length)
labels_n = np.zeros(length)

udp_packet = 0
tcp_packet = 0
other = 0
total_packet = 0
 
i=0
for ts, buf in normal:
    eth = dpkt.ethernet.Ethernet(buf)
    total_packet += 1 

    if eth.type == 2048:
        ip = eth.data
        ips_n[i] = int.from_bytes(ip.src,"little")
        protocol_n[i] = ip.p
        total_length_n[i] = ip.len
        
        if ip.p == 6:
            tcp_packet += 1
            TCP=ip.data 
            dst_port_n[i] = TCP.dport
            src_port_n[i] = TCP.sport
            
            flags_n[i] = TCP.flags
            urgent_pointer_n[i] = TCP.urp
            window_size_n[i] = TCP.win
            header_length_n[i] = TCP.off
        elif ip.p == 17:
            udp_packet += 1
#             UDP=ip.data
# #             dst_port_n[i] = UDP.dport
# #             src_port_n[i] = UDP.sport
#             udp_length_n[i] = UDP.ulen
        else:
            other += 1
#             dst_port_n[i] = 0
#             src_port_n[i] = 0
    else:
        other += 1
        
    i += 1

input.close()
print("total # of packets: %s"  % (total_packet))
print("# of UDP packets: %s" % (udp_packet))
print("# of TCP packets: %s" % (tcp_packet))
print("# of other packets except for TCP or UDP: %s"  % (other))

end = time.time()
print(end - start)


total # of packets: 633894
# of UDP packets: 0
# of TCP packets: 633894
# of other packets except for TCP or UDP: 0
12.505414962768555


### Get attack keys of DDoS attacks containing only TCP traffic

In [3]:
path_to_json = 'signatures/'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]

tcp_keys=[]

for jf in json_files:
    with open('signatures/' + jf) as f:
        data=json.load(f)
    protocol = data.get("protocol")

    if protocol == "TCP" or protocol == "HTTP":
        tcp_keys.append(jf[:-5])

print("# of TCP DDoS attacks: ", len(tcp_keys))

# of TCP DDoS attacks:  279


### Get AMI score and other stats per TCP DDoS Attack

In [4]:
start = time.time()

counter = 1
AMI_values = {}
most_important_features = []

#iterate over all attack keys with UDP attacks
for attack_key in tcp_keys:
    try:
        input = open('attack_pcaps/' + attack_key + '.pcap', 'rb')
        attack = dpkt.pcap.Reader(input)

        udp_packet = 0
        tcp_packet = 0
        other = 0
        total_packet = 0
        i = 0
        
        ############### 
        #loop trough packets to get the number of packets in the attack
        #############
        for ts, buf in attack:
            eth = dpkt.ethernet.Ethernet(buf)
            total_packet += 1 

        input.close()
        
        print("attackkey: ", attack_key)
        print("total # of packets: %s"  % (total_packet))
        #######################
        #initialize numpy arrays for all the to measure features
        ##############################
        ips_a = np.empty(total_packet)
        protocol_a = np.empty(total_packet)
        total_length_a = np.empty(total_packet)
        src_port_a = np.empty(total_packet)
        dst_port_a = np.empty(total_packet)
        udp_length_a = np.empty(total_packet)
        
        header_length_a = np.empty(total_packet)
        urgent_pointer_a = np.empty(total_packet)
        window_size_a = np.empty(total_packet)
        flags_a = np.empty(total_packet)
        
        labels_a = np.ones(total_packet)    
        
        ###############
        #loop trough packets and gather features
        #########
        input = open('attack_pcaps/' + attack_key + '.pcap', 'rb')
        attack = dpkt.pcap.Reader(input)
        i = 0
        for ts, buf in attack:
            eth = dpkt.ethernet.Ethernet(buf)
            if eth.type == 2048:
                ip = eth.data
                ips_a[i] = int.from_bytes(ip.src,"little")
                protocol_a[i] = ip.p
                total_length_a[i] = ip.len

                if ip.p == 6:
                    tcp_packet += 1
                    TCP=ip.data 
                    dst_port_a[i] = TCP.dport
                    src_port_a[i] = TCP.sport
                    
                    flags_a[i] = TCP.flags
                    urgent_pointer_a[i] = TCP.urp
                    window_size_a[i] = TCP.win
                    header_length_a[i] = TCP.off
                    
                elif ip.p == 17:
                    udp_packet += 1
                    UDP=ip.data
                    udp_length_a[i] = UDP.ulen
                    dst_port_a[i] = UDP.dport
                    src_port_a[i] = UDP.sport
                else:
                    other += 1
                    dst_port_a[i] = 0
                    src_port_a[i] = 0
            else:
                other += 1

            i += 1

        ###############
        #make Numpy arrays with attack features and normal features together
        ################

        ips = np.append(ips_a, ips_n)
        protocol = np.append(protocol_a, protocol_n)
        total_length = np.append(total_length_a, total_length_n)
        src_port = np.append(src_port_a, src_port_n)
        dst_port = np.append(dst_port_a, dst_port_n)
        
        header_length = np.append(header_length_a, header_length_n)
        flags = np.append(flags_a, flags_n)
        window_size = np.append(window_size_a, window_size_n)
        urgent_pointer = np.append(urgent_pointer_a, urgent_pointer_n)
        
        labels = np.append(labels_a, labels_n)

        features = [ips,protocol,total_length,src_port,dst_port,header_length,flags,window_size,urgent_pointer]


        
        ##########
        #Calculate adjusted mutual information score
        ############
        scores = []
        for x in features:
            score = adjusted_mutual_info_score(x, labels, "arithmetic")
            scores.append(score)
        AMI_values[attack_key] = scores
        ###########
        #Print all stats per attack
        #######
        print("# of UDP packets: %s" % (udp_packet))
        print("# of TCP packets: %s" % (tcp_packet))
        print("# of other packets except for TCP or UDP: %s"  % (other))
        print("Features: ips,protocol,total_length,src_port,dst_port,header_length,flags,window_size,urgent_pointer")
        print("AMI scores: ", scores)
        
        fname = "signatures/" + attack_key + ".json"
        with open(fname) as jsonfile:
            sig = json.load(jsonfile)
        sig['AMI'] = scores
        with open(fname, mode='w') as f:
            f.write(json.dumps(sig, indent=2))
        
        
        max_index, max_value = max(enumerate(scores), key=operator.itemgetter(1))
        most_important_features.append(max_index)
        print("Most important feature: ", max_index)
        end = time.time()
        print(end - start)
        print(counter,"/",len(tcp_keys))
        counter += 1
        print("#####################################")
    except Exception as e:
        print("something went wrong here ", attack_key)
        print(e)
print(end - start)

attackkey:  cb64829d271436ed8cecb4bb27361bc3
total # of packets: 7762
# of UDP packets: 0
# of TCP packets: 7762
# of other packets except for TCP or UDP: 0
Features: ips,protocol,total_length,src_port,dst_port,header_length,flags,window_size,urgent_pointer
AMI scores:  [0.0235601172790421, 0.0, 0.0072005347023562, 0.015730145020467892, 0.028056029577499315, 0.00786931504150907, 0.11183661044693294, 0.013740590047591034, 0.9997103786594993]
Most important feature:  8
2.8155505657196045
1 / 279
#####################################
attackkey:  5227726708b41c2e32339a1390cee196
total # of packets: 1863628
# of UDP packets: 0
# of TCP packets: 1800265
# of other packets except for TCP or UDP: 63363
Features: ips,protocol,total_length,src_port,dst_port,header_length,flags,window_size,urgent_pointer
AMI scores:  [0.0640905780079912, 0.022020731615289446, 0.3103176735717368, 0.3196400691694169, 0.41059282308005474, 0.15059879469655948, 0.6489868921229536, 0.43142478579376525, 8.82242068406918

# of UDP packets: 0
# of TCP packets: 11911
# of other packets except for TCP or UDP: 0
Features: ips,protocol,total_length,src_port,dst_port,header_length,flags,window_size,urgent_pointer
AMI scores:  [0.03249254375832533, 6.039161208571824e-16, 0.011130507567820361, 0.022013382792258646, 0.039503751837067065, 0.011457365864546038, 0.15112643582625737, 0.020848147150940403, 0.9996064750566301]
Most important feature:  8
288.9698534011841
16 / 279
#####################################
attackkey:  76e4f2ada11069b1817bad8e65eb71e7
total # of packets: 2072
# of UDP packets: 0
# of TCP packets: 2072
# of other packets except for TCP or UDP: 0
Features: ips,protocol,total_length,src_port,dst_port,header_length,flags,window_size,urgent_pointer
AMI scores:  [0.008027335054768301, 4.85506719334583e-13, 0.009274002939043447, 0.004790683374560369, 0.002797212854662102, 0.014217461220207759, 0.008156894057475326, 0.005385735559364005, 4.85506719334583e-13]
Most important feature:  5
291.264345169

attackkey:  c19e1b67baec3afe060c93ef25aee2c4
total # of packets: 8439
# of UDP packets: 0
# of TCP packets: 8439
# of other packets except for TCP or UDP: 0
Features: ips,protocol,total_length,src_port,dst_port,header_length,flags,window_size,urgent_pointer
AMI scores:  [0.025121001708776604, 1.5247631955480442e-13, 0.00784680953048805, 0.01683750006191623, 0.03002044418110238, 0.008478605040170078, 0.11881015229892729, 0.014926823211466699, 1.0]
Most important feature:  8
571.1823761463165
31 / 279
#####################################
attackkey:  16d4cc2995ab2c68bb2f14224222179e
total # of packets: 33524
# of UDP packets: 0
# of TCP packets: 33523
# of other packets except for TCP or UDP: 1
Features: ips,protocol,total_length,src_port,dst_port,header_length,flags,window_size,urgent_pointer
AMI scores:  [0.06403127621447445, 4.199818694975864e-05, 0.030163042422228552, 0.04469836918739233, 0.08551595634200974, 0.02626570792643148, 0.2850495049610344, 0.05324972305781016, 0.99949908770

attackkey:  54d21d48c65ec87809c0adbe71109388
total # of packets: 622650
# of UDP packets: 0
# of TCP packets: 622650
# of other packets except for TCP or UDP: 0
Features: ips,protocol,total_length,src_port,dst_port,header_length,flags,window_size,urgent_pointer
AMI scores:  [0.07467092583665, -7.894715576667383e-15, 0.23186404976297653, 0.10208813294632453, 0.3102143665907169, 0.11948097174821848, 0.6238744712089633, 0.34874870919476325, 1.4965755341492285e-05]
Most important feature:  6
833.7181572914124
46 / 279
#####################################
attackkey:  e5859c5cf5e10220cbb26664259b71d8
total # of packets: 5834
# of UDP packets: 0
# of TCP packets: 5834
# of other packets except for TCP or UDP: 0
Features: ips,protocol,total_length,src_port,dst_port,header_length,flags,window_size,urgent_pointer
AMI scores:  [0.018843402332765193, 6.70296181827045e-14, 0.005352445559710351, 0.012555508582973, 0.022204920898382163, 0.006076293279815503, 0.09058185422051308, 0.010331193706979424

attackkey:  f2d8a69a5d312683681935bf1a34c581
total # of packets: 5013243
# of UDP packets: 0
# of TCP packets: 5013243
# of other packets except for TCP or UDP: 0
Features: ips,protocol,total_length,src_port,dst_port,header_length,flags,window_size,urgent_pointer
AMI scores:  [0.005494793803016904, 1.106445765918143e-15, 0.415134959746289, 0.04286756844574958, 0.5182396909255264, 0.18776775399986365, 0.8542553576218382, 0.5528361017241132, 1.106445765918143e-15]
Most important feature:  6
1552.0036220550537
61 / 279
#####################################
attackkey:  bccff1c4830d567597106c927ef7ad10
total # of packets: 99996
# of UDP packets: 0
# of TCP packets: 99996
# of other packets except for TCP or UDP: 0
Features: ips,protocol,total_length,src_port,dst_port,header_length,flags,window_size,urgent_pointer
AMI scores:  [0.10113868297909111, 1.1155047109427906e-15, 0.07656976838154549, 0.07645931710186418, 0.11351559871627163, 0.05382192434782956, 0.2173146377187136, 0.153334013158754

...

# of UDP packets: 0
# of TCP packets: 35
# of other packets except for TCP or UDP: 0
Features: ips,protocol,total_length,src_port,dst_port,header_length,flags,window_size,urgent_pointer
AMI scores:  [0.00018784642242590014, 0.0, 8.651689754695719e-05, 0.00012743352085141653, 1.2949076552732946e-05, 0.00026280793525791733, 0.00010407816206845734, 8.339756645396545e-05, 0.0]
Most important feature:  5
5066.934689044952
211 / 279
#####################################
attackkey:  36f8b65846f7954d2b87505718997b63
total # of packets: 2397636
# of UDP packets: 0
# of TCP packets: 2397634
# of other packets except for TCP or UDP: 2
Features: ips,protocol,total_length,src_port,dst_port,header_length,flags,window_size,urgent_pointer
AMI scores:  [0.019251651269156827, -1.2587158103396707e-07, 0.3532084706940594, 0.06553993291219733, 0.45562825860117767, 0.1620158500148629, 0.6706779972361089, 0.4857928294727755, 1.9646470010499096e-05]
Most important feature:  6
5161.331468582153
212 / 279
#####

attackkey:  9a1340fc25c5c634a95cd3bb3ea3cdac
total # of packets: 1632452
# of UDP packets: 0
# of TCP packets: 1632452
# of other packets except for TCP or UDP: 0
Features: ips,protocol,total_length,src_port,dst_port,header_length,flags,window_size,urgent_pointer
AMI scores:  [0.028794726860404955, 0.0, 0.3240034349823427, 0.07812041744397755, 0.4179007830800101, 0.15339717841639033, 0.6981903419397989, 0.452761326982689, 7.704785462356854e-06]
Most important feature:  6
5436.644411802292
226 / 279
#####################################
attackkey:  8c57fb5e3a653289624f17abb8dcaac4
total # of packets: 628778
# of UDP packets: 0
# of TCP packets: 628778
# of other packets except for TCP or UDP: 0
Features: ips,protocol,total_length,src_port,dst_port,header_length,flags,window_size,urgent_pointer
AMI scores:  [0.07419358022816341, -2.5627715514378187e-15, 0.23287914236920707, 0.1019762442615645, 0.3113720435100011, 0.11993979180181562, 0.6249974366951214, 0.34990164149058234, 2.27461464355

attackkey:  7fc0cd836b1f8b32bae78b6cd7cacf46
total # of packets: 12957
# of UDP packets: 0
# of TCP packets: 12957
# of other packets except for TCP or UDP: 0
Features: ips,protocol,total_length,src_port,dst_port,header_length,flags,window_size,urgent_pointer
AMI scores:  [0.035180337978223, 0.0, 0.024306992639232094, 0.02365721961604032, 0.019111345004312965, 0.09322785451627799, 0.006761190178352559, 0.02343778773038056, 0.0]
Most important feature:  5
5535.0256633758545
241 / 279
#####################################
attackkey:  adf03baad2a313a99ac0b30dc09c4f27
total # of packets: 25887
# of UDP packets: 0
# of TCP packets: 25887
# of other packets except for TCP or UDP: 0
Features: ips,protocol,total_length,src_port,dst_port,header_length,flags,window_size,urgent_pointer
AMI scores:  [0.054958639544900545, 0.0, 0.02371758168898573, 0.038043597177441296, 0.07111130056299703, 0.021601483331995377, 0.24673176227305102, 0.04260594238121697, 0.9997789754316776]
Most important feature:  

attackkey:  d0cde5bd4442d9fa6e88e6c35e4ce761
total # of packets: 99994
# of UDP packets: 0
# of TCP packets: 99994
# of other packets except for TCP or UDP: 0
Features: ips,protocol,total_length,src_port,dst_port,header_length,flags,window_size,urgent_pointer
AMI scores:  [0.101138049982521, 0.0, 0.07656295614905827, 0.07647466790281793, 0.11351395818594713, 0.053803700105926375, 0.4343326645359537, 0.15335362736993252, 0.0]
Most important feature:  6
5958.705632925034
256 / 279
#####################################
attackkey:  7638c8061f9cb0dc0c3f7c614ac52725
total # of packets: 1417111
# of UDP packets: 0
# of TCP packets: 1417111
# of other packets except for TCP or UDP: 0
Features: ips,protocol,total_length,src_port,dst_port,header_length,flags,window_size,urgent_pointer
AMI scores:  [0.033214582860085225, -1.7954589048091793e-15, 0.31208241786191043, 0.08257697611832401, 0.40316114966834093, 0.14940908968384686, 0.7044940023585571, 0.4393471860596608, -1.6513127965351226e-07]
Most

attackkey:  b7f9552d3ed992c211e581de166c96f2
total # of packets: 88600
# of UDP packets: 0
# of TCP packets: 88600
# of other packets except for TCP or UDP: 0
Features: ips,protocol,total_length,src_port,dst_port,header_length,flags,window_size,urgent_pointer
AMI scores:  [0.12714758822591812, -8.35350689805877e-15, 0.1410133281861851, 0.07559112521000876, 0.10387720319105669, 0.3118369999762955, 0.15297113843662258, 0.10914065323009903, -8.35350689805877e-15]
Most important feature:  5
6217.09163069725
271 / 279
#####################################
attackkey:  d41d698fd2ae26307528cd4a0c6de5e8
total # of packets: 1337377
# of UDP packets: 0
# of TCP packets: 1337377
# of other packets except for TCP or UDP: 0
Features: ips,protocol,total_length,src_port,dst_port,header_length,flags,window_size,urgent_pointer
AMI scores:  [0.03871625089452296, 5.6567880230936385e-15, 0.30363486616636426, 0.08439564769656505, 0.39700192961446457, 0.14507471887332443, 0.6512011433617658, 0.43052784137656

In [42]:
with open('data.json', 'w') as json_file:
    json.dump(AMI_values, json_file)

NameError: name 'AMI_values' is not defined

In [16]:
for i in range(0,9):
    print(i,most_important_features.count(i))

0 1
1 0
2 0
3 0
4 1
5 27
6 100
7 2
8 141


### Analysis and Conclusions

ips,protocol,total_length,src_port,dst_port,header_length,flags,window_size,urgent_pointer <br/>

0 1 <br/>
1 0 <br/>
2 0 <br/>
3 0 <br/>
4 1 <br/>
5 Header length: 30 <br/>
6 Flags: 103 <br/>
7 2 <br/>
8 Urgent pointer: 141 <br/>

The urgent pointer inside the DDoS attack is the most iportant feature. If we remove this feature we'll see that the TCP flags become the most important feature.

In [41]:
import json
import numpy as np
import operator
with open('tcp_data.json') as json_file:  
    data = json.load(json_file)

l = []
for p in data:
    array = np.array(data[p])
    i = array.argmax()
    if i == 8:
        array = np.delete(array,i)
    i = array.argmax()
    l.append(i)

for i in range(0,9):
    print(i,l.count(i))

0 1
1 0
2 0
3 0
4 1
5 27
6 241
7 2
8 0
